<body>
    <font size="2">Florian Schnabel, 11807470, SS 2023</font><br />
</body>

# Exercise 2 - Conduction and Convection with RC-Networks

This script aims to calculate the temperatures of an arbitrary number of nodes in a steady state. The nodes themselves are connected through conductances representing convection and thermal heat transfer {numref}`fig:RC-Networks`.

```{figure} ./Figures/RC-Networks.png
---
width: 350px
name: fig:RC-Networks
---
Conveductance network with three nodes {cite}`hagenthoft2001`
```

 To allow a numerical solution of the problem, the partial differential equation for heat conduction must be discretised. The equation for heat conduction is shown in {eq}`heatflow` {cite}`walther2021`:
 
$$\rho \cdot c \cdot \frac{dT}{dt} = \nabla (\lambda \nabla T) $$ (heatflow)

reduced to one dimension:

$$ \rho \cdot c \cdot \frac{dT}{dt} =  \frac{d}{dx}(\lambda \frac{dT}{dx}) $$


## Space discretisation

The discretisation is done using the finite volume method and a representation as RC-Networks. Each cell is represented by a resistance and the conductivity to the neighbouring cells. The conductivity between Interrior cells is calculated as follows (for simplicity, the surface resistance between layers is neglected. $ R_{ci} =0 $ ) {cite}`hagenthoft2001`:

$$K_{i-0.5} = \frac{1}{\frac{0.5 \Delta x_{i-1}}{\lambda_{i-1}} + R_{ci} + \frac{0.5 \Delta x_{i}}{\lambda_{i}}} $$ (conductivityInterior)

For a cell inside a layer ($\lambda_{i-1} = \lambda_{i} = \lambda_{i+1}$ and $\Delta x_{i-1} = \Delta x_{i} = \Delta x_{i+1}$) {eq}`conductivityInterior` collapses to {eq}`conductivityInteriorcoll`:

$$ K_{i-0.5} = K_{i+0.5} = \frac{\lambda_{i}}{\Delta x_{i}} $$ (conductivityInteriorcoll)


And for Cells at the boundary:

$$ K_{BC,1} = K_{0.5} = \frac{1}{R_{s,1} + \frac{0.5 \Delta x_{1}}{\lambda_{1}}} $$

## Time Integration
Using the explicit euler sheme the next Timestep can be calculated as follows:

 
$$
\begin{aligned}
    &\text{for cells at the left boundary:}& &T_1^{n+1} = T_1^n+F_{o,1}^* \cdot (K_{BC,1} \cdot   T_{BC}^n  - (K_{BC,l}+K_{2}) \cdot   T_{1}^n + K_{2} \cdot   T_{2}^n)& \\
    &\text{for interior cells:}& &T_i^{n+1} = T_1^n+F_{o,i}^* \cdot (K_{i-1} \cdot   T_{i-1}^n  - (K_{i-1}+K_{i+1}) \cdot   T_{i}^n + K_{i+1} \cdot   T_{i+1}^n)& \\
    &\text{for cells at the right boundary:}& &T_j^{n+1} = T_j^n+F_{o,j}^* \cdot (K_{j-1} \cdot   T_{j-1}^n  - (K_{j-1}+K_{BC,2}) \cdot   T_{j}^n + K_{BC,2} \cdot   T_{BC,2}^n)& \\
\end{aligned}
$$


or generally written (based on {cite}`walther2021`):

$$ T^{n+1} = T^n + F_{o}^* \cdot K \cdot T^n $$ (timeint)



With the conductivity matrix $ K $:



$$ K = \left [ 
\begin{array}{ccccccc}
    0 & 0 & 0 & 0 & (...) & 0 & 0 & 0  & 0\\
    K_{BC,1} & -(K_{BC,1} + K_w) & K_w & 0 & (...) & 0 & 0 & 0 & 0\\
    0 & K_e & -(K_e + K_w) & K_w & (...) & 0 & 0  & 0 & 0\\
    (...) & (...) & (...) & (...) & (...) & (...) & (...)  & (...) & (...)\\
    0 & 0 & 0 & 0 &(...) & K_{e} & -(K_e + K_w) & K_w & 0 \\
    0 & 0 & 0 & 0 &(...) & 0 & K_{e} & -(K_e + K_{BC,2}) & K_{BC,2} \\
    0 & 0 & 0 & 0 &(...) & 0 & 0 & 0 & 0 \\
\end{array}
\right] $$

and the adapted "resistance" matrix $F_{o}^*$ (example for layers a,b,c):

$$ F_{o}^* = \left [ 
\begin{array}{ccccccc}
    F_{o,a}^* & 0\\
    0 & F_{o,a}^*  & 0\\ 
     &  0 &  (...)  & 0\\
      &   &  0 & F_{o,b}^* & 0\\
      &   &    & 0 & F_{o,b}^* & 0\\
      &   &   &    & 0  &  (...) & 0\\
      &   &   &   &    & 0 &  F_{o,c}^* & 0\\
\end{array}
\right]  \quad F_{o,i}^* = \frac{\Delta t}{\Delta x_i} \cdot \frac{1}{\rho_i \cdot c_i}$$

with the heat capacity $c_i$ and the density $\rho_i$.

### Stability

{eq}`stability` shows a condition to ensure $F_o^* \cdot K$ results in a diagonally dominant (invertible) matrix  as a function of the Fourier’s number {cite}`walther2021`.

$$ (1-2 F_o) \geq 0 \longrightarrow F_o = \frac{\lambda}{\rho \cdot c} \cdot \frac{\Delta t}{\Delta x^2} \le 0.5 $$ (stability)

To insure a stable simulation this condition is tested during the construction of the conductivity matrix $K$.

### Steady state criteria

The simulation stops once a steady state is achieved. The ISO 10211 {cite}`ISO10211` norm gives a steady state criterea for thermal bridges in building constructions. For iterative solvers the following limit is defined:

$$ \sum q_{in} - \frac{\sum q}{2} < 0.0001 $$

As derived in the exercise the heatflow through the surface of a finite volume can be approximated as follows:

$$
\begin{aligned}
q_{e} &= \lambda_e \cdot \frac{T_E-T_P}{\Delta x_e} \\
q_{w} &= \lambda_w \cdot \frac{T_P-T_W}{\Delta x_w}
\end{aligned}
$$ (flows)


Specifing for the first and last cells with $\Delta x /2$ and $R_{se}$ / $R_{si}$ the heatflow in and out of the construction can be written as:

$$
\begin{aligned}
 q_{in} = K_{BC1} \cdot (T_{BC1}-T_{1}) \\
 q_{out} = K_{BC2} \cdot (T_{BC2}-T_{n}) 
\end{aligned}
$$

## Implementation

In [1]:
%reset

#Libraries
import numpy as np
import matplotlib.pyplot as plt

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [2]:
class node:
    # n summarizes the number of all nodes
    n=0
    def __init__(self, number):
        self.number = number
        node.n += 1
    

In [3]:
class convection:
    def __init__(self, fromNode, toNode, massflow):
        self.fromNode = fromNode
        self.toNode = toNode
        self.massflow = massflow

In [4]:
class conduction:
    def __init__(self, fromNode, toNode, conductance):
        self.fromNode = fromNode
        self.toNode = toNode
        self.K = conductance
        

In [5]:
class boundary:
    def __init__(self, toNode, temp, conductance):
        self.toNode = toNode
        self.temp = temp
        self.K = conductance

In [6]:
class heatSource:
    def __init__(self, toNode, power):
        self.toNode = toNode
        self.power = power

### Construction of the node, convection and conduction objects
Input Values according to the excersise description with the following units:
* [$T$]= $^\circ C$
* [$I$]= $W C$
* [$\overset{.}{M}$]= $kg s^{-1}$


In [7]:
nodes = []
nodes.append(node(1))
nodes.append(node(2))
nodes.append(node(3))

boundaries = []
boundaries.append(boundary(1,19,0.1))
boundaries.append(boundary(2,25,0.1))
boundaries.append(boundary(3,0,0.1))

convections = []
convections.append(convection(1,2,0.05))
convections.append(convection(3,2,0.05))
convections.append(convection(1,3,0))

conductions = []
conductions.append(conduction(1,2,0.1))
conductions.append(conduction(2,3,0.1))
conductions.append(conduction(1,3,0.1))

In [8]:
heatSources = []
heatSources.append(heatSource(1,10))
heatSources.append(heatSource(2,20))
heatSources.append(heatSource(3,10))

In [9]:
ca = 1000 #J/kgK

## Variant 1 - without convection

### Construction of the $K$ matrix and the boundary vector $I_0$

In [10]:
# reference number of nodes to increase readability
n = nodes[0].n

# Initialising conductivity matrix K
K = np.zeros((n,n))

# Initialising boundary vector I0
I0 = np.zeros((n))

# iterater to keep track of cell numbers
counter = 1
for node in nodes:
    #initialize empty conductivity matrix for the current layer
    
    I0i=0
    Ki=0
    
    for boundary in boundaries:
        if boundary.toNode == node.number:
            Ki += boundary.K
            I0i += boundary.K*boundary.temp
        
    for heatSource in heatSources:
        if heatSource.toNode == node.number:
            I0i += heatSource.power
            
    for conduction in conductions:
        if conduction.toNode == node.number or conduction.fromNode == node.number:
            Ki += conduction.K
            
    for conduction in conductions:
        if conduction.toNode == node.number:
            K[node.number-1, conduction.fromNode-1]= conduction.K
            K[conduction.fromNode-1, node.number-1]= conduction.K
    
    #for conduction in conductions:
    #    if conduction.fromNode == node.number:
    #        K[node.number-1, conduction.toNode-1]= conduction.K
    
    #for convection in convections:
    #    if convection.fromNode == node.number:
    #        Ki += conveductance.K + conveductance.massflow * ca
    #        K[conveductance.toNode-1 , node.number-1]= conveductance.K + conveductance.massflow * ca
    #        K[node.number-1 , node.number-1]= conveductance.K + conveductance.massflow * ca

   
    #defining the diagonal axis
    K[node.number-1, node.number-1]= - Ki
    
    #defining the I0
    I0[node.number-1]= I0i

In [11]:
print(K)
print(conductions[2].fromNode-1,conductions[2].toNode-1)
print(I0)
print(n)

[[-0.3  0.1  0.1]
 [ 0.1 -0.3  0.1]
 [ 0.1  0.1 -0.3]]
0 2
[11.9 22.5 10. ]
3


### Calculating the temperature field

In [12]:
T=np.linalg.solve(K,-I0)

In [ ]:
print(T)

[140.75 167.25 136.  ]


In [14]:
#plt.xlabel("x position [m]")
#plt.ylabel("Temperature [°C]")
#plt.plot(x_pos, T_plus, 'o-', alpha=0.65)
#x_w=0
#for layer in layers:
#    plt.plot([x_w,x_w], [-5,35], color='k', alpha=0.7)
#    x_w += layer.width
#plt.plot([layer.width_sum,layer.width_sum], [-5,35], color='k', alpha=0.7)
#plt.title("Steady state temperature field in the building Component")
#a = np.array([1,-1,3])

## Variant 2 - without convection

### Alteration of the $K$ matrix and the boundary vector $I_0$

In [15]:
for node in nodes:
    for convection in convections:
        if convection.fromNode == node.number:
            K[node.number-1 , node.number-1] += -convection.massflow * ca
            K[convection.toNode-1 , node.number-1] += convection.massflow * ca

In [16]:
print(K)
print(conductions[2].fromNode-1,conductions[2].toNode-1)
print(I0)
print(n)

[[-50.3   0.1   0.1]
 [ 50.1  -0.3  50.1]
 [  0.1   0.1 -50.3]]
0 2
[11.9 22.5 10. ]
3


### Calculating the temperature field

In [17]:
T=np.linalg.solve(K,-I0)

In [18]:
print(T)

[  1.11706349 441.80357143   1.07936508]
